In [3]:
import pandas as pd
import numpy as np
import os
import re

# --- Define output_dir HERE ---
output_dir = "citibike_feb_data"
output_cleaned_dir = "citibike_feb_data_cleaned"  # New output directory
# ---

# Schema mapping for different years
schema_mapping = {
    "pre_2017": {
        "tripduration": "trip_duration",
        "starttime": "start_time",
        "stoptime": "end_time",
        "start station id": "start_station_id",
        "start station name": "start_station_name",
        "start station latitude": "start_station_latitude",
        "start station longitude": "start_station_longitude",
        "end station id": "end_station_id",
        "end station name": "end_station_name",
        "end station latitude": "end_station_latitude",
        "end station longitude": "end_station_longitude",
        "bikeid": "bike_id",
        "usertype": "user_type_clean",
        "birth year": "birth_year",
        "gender": "gender",
    },
    "2017": {
        "Start Time": "start_time",
        "Stop Time": "end_time",
        "Trip Duration": "trip_duration",
        "Start Station ID": "start_station_id",
        "Start Station Name": "start_station_name",
        "Start Station Latitude": "start_station_latitude",
        "Start Station Longitude": "start_station_longitude",
        "End Station ID": "end_station_id",
        "End Station Name": "end_station_name",
        "End Station Latitude": "end_station_latitude",
        "End Station Longitude": "end_station_longitude",
        "Bike ID": "bike_id",
        "User Type": "user_type_clean",
        "Birth Year": "birth_year",
        "Gender": "gender",
    },
    "pre_2020": {
        "tripduration": "trip_duration",
        "starttime": "start_time",
        "stoptime": "end_time",
        "start station id": "start_station_id",
        "start station name": "start_station_name",
        "start station latitude": "start_station_latitude",
        "start station longitude": "start_station_longitude",
        "end station id": "end_station_id",
        "end station name": "end_station_name",
        "end station latitude": "end_station_latitude",
        "end station longitude": "end_station_longitude",
        "bikeid": "bike_id",
        "usertype": "user_type_clean",
        "birth year": "birth_year",
        "gender": "gender",
    },
    "post_2020": {
        "ride_id": "ride_id",
        "rideable_type": "rideable_type",
        "started_at": "start_time",
        "ended_at": "end_time",
        "start_station_name": "start_station_name",
        "start_station_id": "start_station_id",
        "end_station_name": "end_station_name",
        "end_station_id": "end_station_id",
        "start_lat": "start_station_latitude",
        "start_lng": "start_station_longitude",
        "end_lat": "end_station_latitude",
        "end_lng": "end_station_longitude",
        "member_casual": "user_type_clean",
    },
}


# Function to determine the file type based on the filename
def get_file_type(filename):
    match = re.match(r"(\d{4})\d{2}-", filename)
    if match:
        year = int(match.group(1))
        if year == 2017:
            return "2017"
        elif year >= 2020:
            return "post_2020"
        else:
            return "pre_2020"
    return "Unknown"


# Create the output directory if it doesn't exist
os.makedirs(output_cleaned_dir, exist_ok=True)

# Initialize a global trip ID counter
global_trip_id = 1

# Iterate through extracted CSV files *INSIDE* the year/month subdirectories
for year in range(2014, 2024):
    year_dir = os.path.join(output_dir, str(year))
    month_dir = os.path.join(year_dir, "02")  # Directly go to the February folder

    if os.path.isdir(month_dir):  # Check if the month directory exists
        # --- Group files by the common prefix ---
        file_groups = {}
        for filename in os.listdir(month_dir):
            if filename.endswith(".csv"):
                # Extract the common prefix (e.g., "201402-citibike-tripdata")
                prefix = re.match(r"(.+?)_\d+\.csv", filename)
                if not prefix:  # Handle files without the _1, _2 suffix
                    prefix = re.match(r"(.+?)\.csv", filename)  # extract prefix even if there is no underscore
                prefix_str = prefix.group(1) if prefix else filename.replace(".csv", "")

                if prefix_str not in file_groups:
                    file_groups[prefix_str] = []
                file_groups[prefix_str].append(filename)

        # --- Process each group of files ---
        for prefix, filenames in file_groups.items():
            combined_df = pd.DataFrame()  # Initialize an empty DataFrame

            for filename in filenames:
                filepath = os.path.join(month_dir, filename)
                file_type = get_file_type(filename)
                print(f"Processing {filename} (Type: {file_type})...")

                # Load the CSV, handling potential errors
                try:
                    df = pd.read_csv(filepath, low_memory=False)
                except pd.errors.ParserError as e:
                    print(f" Error reading {filename}: {e}. Skipping.")
                    continue  # Skip this file and move to the next
                except Exception as e:
                    print(
                        f" An unexpected error occurred reading {filename}: {e}. Skipping"
                    )
                    continue  # Skip this file and move to the next
                
                # Apply schema mapping
                mapping = schema_mapping.get(file_type, {})
                df = df.rename(columns=mapping)

                # Ensure start_time and end_time are datetime objects
                if "start_time" in df.columns:
                    df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
                    df["start_time"] = df["start_time"].dt.strftime('%m/%d/%Y %H:%M')  #Format the date.
                    df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce") #Convert back to datetime to check for and remove NaT.

                if "end_time" in df.columns:
                    df["end_time"] = pd.to_datetime(df["end_time"], errors="coerce")
                    df["end_time"] = df["end_time"].dt.strftime('%m/%d/%Y %H:%M')
                    df["end_time"] = pd.to_datetime(df["end_time"], errors="coerce")
                
                # Add the global trip ID, and increment the counter.
                num_rows = len(df)
                df["trip_id"] = range(global_trip_id, global_trip_id + num_rows)
                global_trip_id += num_rows

                #Ensure trip_start_year column exists before filtering
                if "start_time" in df.columns:  # Use the correctly formatted start_time
                     df["trip_start_year"] = df["start_time"].dt.year
                     df["trip_start_month"] = df["start_time"].dt.month

                # Ensure no data loss from filtering step
                if (
                    "trip_start_month" in df.columns
                    and "trip_start_year" in df.columns
                ):
                   df = df[df["trip_start_month"] == 2]  # Keep February
                   
                combined_df = pd.concat([combined_df, df], ignore_index=True)

            # --- Output the combined DataFrame ---
            if not combined_df.empty:  # Check if df is not empty.
                output_filename = f"{prefix}_cleaned.csv"
                output_path = os.path.join(output_cleaned_dir, output_filename)
                combined_df.to_csv(output_path, index=False)
                print(f"Cleaned data exported to {output_path}")

Processing 201402-citibike-tripdata_1.csv (Type: pre_2020)...
Cleaned data exported to citibike_feb_data_cleaned\201402-citibike-tripdata_cleaned.csv
Processing 201502-citibike-tripdata_1.csv (Type: pre_2020)...
Cleaned data exported to citibike_feb_data_cleaned\201502-citibike-tripdata_cleaned.csv
Processing 201602-citibike-tripdata_1.csv (Type: pre_2020)...
Cleaned data exported to citibike_feb_data_cleaned\201602-citibike-tripdata_cleaned.csv
Processing 201702-citibike-tripdata.csv_1.csv (Type: 2017)...
Cleaned data exported to citibike_feb_data_cleaned\201702-citibike-tripdata.csv_cleaned.csv
Processing 201802-citibike-tripdata.csv (Type: pre_2020)...
Processing 201802-citibike-tripdata_1.csv (Type: pre_2020)...
Cleaned data exported to citibike_feb_data_cleaned\201802-citibike-tripdata_cleaned.csv
Processing 201902-citibike-tripdata_1.csv (Type: pre_2020)...
Cleaned data exported to citibike_feb_data_cleaned\201902-citibike-tripdata_cleaned.csv
Processing 202002-citibike-tripdata_